### 物件稱為DStream -離散物件->一定時間內到達的資料 ->,每個DStream 代表著每個時間刻度所到達的一串RDD

In [ ]:
HDFS
Flume => Spark Stream(to build a DStream) => 轉換 或是 輸出
Kafka

In [ ]:
import org.apache.spark.streaming.StreamingContext
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.dstream.DStream
import org.apache.spark.streaming.Duration
import org.apache.spark.streaming.Seconds
import org.apache.spark.SparkConf

In [ ]:
#透過SparkConf建立StreamingContext ,並設定批次間隔為1秒
val ssc = new StreamingContext(conf, Seconds(1))

#local machine
val lines = ssc.socketTextStream("localhost", 7777)

val errorLines = lines.filter(_.contains("error"))

errorLines.print()

In [ ]:
ssc.start() #let Job give to SparkContext
ssc.awaitTermination() #wait util the job is finish

# 串流Context只能使用被啟動一次,而且必須完成DStream與輸出操作後才能夠啟動

In [ ]:
spark-submit --class com.oreilly.lear..StreamingLogInput ASSEMBLY_JAR local[4]

In [ ]:
nc localhost 7777

## SparkStreaming build a checkpoint 

## 週期性的資料放到可信賴的系統(HDFS or S3),if loss data will check from nearly checkpoint and cal again

In [ ]:
DStreams => 非具態(stateless) and 具態(stateful)

### 非具態(stateless):會獨立處理每個批次的資料,與先前的批次資料沒有相依關係 map() filter() reduceByKey()

### 具態(stateful):使用先前批次中的資料或是中間結果來計算目前的批次資料並產出結果。會基於 slide windows or 時間變化的狀態

In [ ]:
stateless: map(),flatMap(),filter(),repartition(),reduceByKey(),groupByKey()

In [ ]:
stateful:

ssc.checkpoint("hdfs://...")

# windows
val accessLogsWindow = accessLogsDStream.window(Seconds(30), Seconds(10))
val windowCounts = accessLogsWindow.count()

In [1]:
### 計算每個IP的訪問次數
val ipDStreaming = accessLogsDStream.map(logEntry.getIpAddress(),1))
val ipCountDStream = ipDStream.reduceByKeyAndWindow(
{(x, y) => x +y}, //將進入視窗的新批次加入
{(x, y) => x -y}, //將離開視窗的舊批次資料移除
Seconds(30), //視窗長度
Seconds(10)  //滑動長度
)

In [ ]:
### scala updateStateByKey() 對HTTP 回應碼計數
def updateRunningSum(values: Seq[Long], state: Option[Long]) = {
    Some(state.getOrElse(0L) + values.size)
}
val responseCodeStream = accessLogsDStream.map(log => (log.getResponseCode(),1L))
val responseCodeCountDStream = responseCodeDStream.updateStateByKey(updateRunningSum _)

### Export

In [ ]:
ipAddressRequestCount.saveAsTextFiles("outputDir", "txt")

In [ ]:
ipAddressRequestCount.foreach{
    rdd => rdd.foreachPartition {
        //start to connect to DB
        partition =>
        partitionn.foreach {
            item =>pass data to outside System
        }
        //close connect
    }
}

### Spark Streaming 要從檔案建立串流時, 要求檔案的目錄名稱必須有一致的日期格式,必且是ACIDS的

In [ ]:
val logData = ssc.textFileStream(logDirectory)

# Kafka

In [ ]:
#訂閱Panda 主題的客戶
import org.apache.spark.streaming.kafka._
...
val topics = List(("pandas",1),("logs",1)).toMap
val topicLines = kafkaUtils.createStream(ssc. ZKQuorum, group, topics)
topicLines.print()

In [ ]:
#讀取Panda主題
import org.apache.spark.streaming.kafka._
import kafka.serializer.StringDecoder

val kafkaParams = Map[String, String]("metadata.broker.list" -> brokers)

val topicSet = List("pandas", "logs").toSet
val topicLines = kafkaUtils.createDirectStream[String, String,
                StringDecoder, StringDecoder](ssc, kafkaParams, topicSet)
StreamingLogInput.processLines(topicLines.map(_._2))

### 全天候運行Spark Streaming

In [ ]:
1. checkpoint
2. 容錯議題
3. unTrust input